In [1]:
import pandas as pd
import os
import json

In [13]:
def fetch_results(dataset_name, model_name, dataset_split):
    results = []
    directory = f"generated_quiz_answers/{dataset_name}/{model_name}/{dataset_split}"

    for language in os.listdir(directory):
        language_directory = os.path.join(directory, language)
        json_file = os.path.join(language_directory, "contamination.json")
        df_file = os.path.join(language_directory, "quiz_answers.csv")
        df_ans = pd.read_csv(df_file)

        with open(json_file) as f:
            data = json.load(f)
            score = data["score"]
            contamination = data["contamination"]

            results.append(
                {
                    "Language": language,
                    f"Score_{model_name}": score,
                    f"Contamination_{model_name}": contamination,
                    f"Total_points_{model_name}": len(df_ans),
                }
            )

    df = pd.DataFrame(results)
    return df

In [14]:
results_udpos_gpt = fetch_results("udpos", "dev-moonshot", "test")

In [15]:
results_udpos_palm = fetch_results("udpos", "palm-32k", "test")

In [16]:
results_udpos_palm

,Language,Score_palm-32k,Contamination_palm-32k,Total_points_palm-32k
0,fr,0.34,0.120000,109
1,zh,0.15,-0.133333,110
2,el,0.16,-0.120000,108
3,bg,0.10,-0.200000,110
4,fi,0.15,-0.133333,110
5,pl,0.09,-0.213333,110
6,it,0.23,-0.026667,106
7,et,0.10,-0.200000,109
8,ro,0.15,-0.133333,109
9,hu,0.13,-0.160000,106


In [17]:

# join the two dataframes on language
results_udpos_combined = results_udpos_gpt.merge(results_udpos_palm, on="Language")

In [18]:
results_udpos_combined

,Language,Score_dev-moonshot,Contamination_dev-moonshot,Total_points_dev-moonshot,Score_palm-32k,Contamination_palm-32k,Total_points_palm-32k
0,fr,0.32,0.093333,110,0.34,0.120000,109
1,zh,0.30,0.066667,110,0.15,-0.133333,110
2,el,0.38,0.173333,110,0.16,-0.120000,108
3,bg,0.29,0.053333,110,0.10,-0.200000,110
4,fi,0.28,0.040000,110,0.15,-0.133333,110
5,pl,0.24,-0.013333,110,0.09,-0.213333,110
6,it,0.27,0.026667,109,0.23,-0.026667,106
7,et,0.37,0.160000,109,0.10,-0.200000,109
8,ro,0.06,-0.253333,110,0.15,-0.133333,109
9,hu,0.40,0.200000,110,0.13,-0.160000,106


In [ ]:
results_